In [ ]:
import pandas as pd
import sqlite3
import numpy as np
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import spacy
import pickle

import wordcloud
import nltk

import sys
sys.path.insert(1, '..')
import utils

import matplotlib.pyplot as plt
#import matplotlib.ticker as ticker
#import seaborn as sns
%matplotlib inline

plt.rcParams["figure.figsize"] = (50,20)
plt.rcParams['figure.dpi'] = 90
plt.rcParams.update({'font.size': 40})
pd.set_option('display.max_colwidth', None)

import warnings
warnings.filterwarnings('ignore')  # "error", "ignore", "always", "default", "module" or "on

DPI = 300

## PEGA DADOS PROCESSADOS 

In [ ]:
df = pd.read_pickle('dados\\df_processado.pkl')
df = df[~df['sent_manual'].isin(['D'])] #remove tweets marcadaos com D
df = df[["data","usuario","tweet","tweet_limpo"]]

#### CLASSIFICA TWEETS

##### Bert Neuralmind

In [ ]:
df_aux = pd.read_pickle('dados\\df_BERT_neuralmind.pkl')
df = df.merge(df_aux, how='left', left_index=True, right_index=True)


In [ ]:
print(df['BERT_neuralmind'].value_counts().sort_values(ascending=False))
print("N {:.2f}".format(df['BERT_neuralmind'].value_counts().sort_values(ascending=False)[0]/df['BERT_neuralmind'].value_counts().sort_values(ascending=False).sum()))
print("N {:.2f}".format(df['BERT_neuralmind'].value_counts().sort_values(ascending=False)[1]/df['BERT_neuralmind'].value_counts().sort_values(ascending=False).sum()))

In [ ]:
df = df.reset_index() 
df = df.drop_duplicates()
df.tweet = df.tweet.str.lower()

## SÉRIE DO COPOM

In [ ]:
df_copom = df[df['tweet'].apply(lambda x: x.lower()).str.contains('|'.join(['copom','selic'])) ]
df_copom = df_copom.loc[df_copom['data']>"2010-01-01"]
# a list of "1" to count the hashtags
ones = [1]*df_copom['data'].shape[0]
# the index of the series
idx = pd.DatetimeIndex(df_copom['data'])
# Resampling / bucketing
s_h_copom = pd.Series(ones, index=idx).resample('H').sum().fillna(0)
df_copom.shape

## PEGA DATAS DO COPOM

In [ ]:
datas = pd.read_csv("base_sqlite3\\datas-copom.csv", header=None)
datas.columns = ["datas"]
datas.datas = pd.to_datetime(datas.datas)
datas.shape

In [ ]:
datas = datas.loc[datas['datas']>"2010-01-01"]

## Pega a hora da divulgação da decisão do Copom

In [ ]:
d = np.datetime_as_string(datas.datas.values, unit = 'h')
d_mais15 = np.datetime_as_string( datas.datas.values + np.timedelta64(15,'h'), unit = 'h')
d_mais24 = np.datetime_as_string( datas.datas.values + np.timedelta64(24,'h'), unit = 'h')

arr_h = np.empty((0, 10))
for i in range(0,(d.shape[0])):
    str2 = str(d_mais24[i])
    str1 = str(d_mais15[i])
    arr_h = np.vstack((arr_h,s_h_copom[str1:str2].values))

In [ ]:
# o índice idx_h representa o número de horas após às 15hs em que se deu o anúcio do copom
idx_h = arr_h.argmax(axis = 1)
#hora_copom = np.datetime_as_string( datas.datas.values + np.timedelta64(15,'h') + np.timedelta64(idx_h,'h') , unit = 'h')
hora_copom = np.array( [datas.datas.values[i] + np.timedelta64(15,'h') + np.timedelta64(idx_h[i],'h')  for i in range(idx_h.shape[0])])


## monta dataframe de impacto

In [ ]:
df_copom.set_index('data', inplace=True)

In [ ]:
d = np.datetime_as_string(hora_copom, unit = 'h')
d_menos1 = np.datetime_as_string( hora_copom - np.timedelta64(1,'h'), unit = 'h')
d_menos24 = np.datetime_as_string( hora_copom - np.timedelta64(24,'h'), unit = 'h')
d_mais24 = np.datetime_as_string( hora_copom + np.timedelta64(24,'h'), unit = 'h')

pre_copom = []
pos_copom = []
pre_copom_tot = []
pos_copom_tot = []


for i in range(d.shape[0]):
    str0 = str(d[i])
    str0_ = str(d_menos1[i])
    str1 = str(d_menos24[i])
    str2 = str(d_mais24[i])
    #pre copom
    total = df_copom.loc[str1:str0_, 'BERT_neuralmind'].value_counts().values
    if total.shape[0] == 2:
        neutros, criticas = total
    elif total.shape[0] ==1:
        neutros = total[0]
        criticas = 0
    else:
        raise("total.shape diferente de 2 e de 1")
    pre_copom.append( criticas/(criticas+neutros) )
    pre_copom_tot.append( criticas+neutros ) 
    #pos copom
    total = df_copom.loc[str0:str2, 'BERT_neuralmind'].value_counts().values
    if total.shape[0] == 2:
        neutros, criticas = total
    elif total.shape[0] ==1:
        neutros = total[0]
        criticas = 0
    else:
        raise("total.shape diferente de 2 e de 1")
    pos_copom.append( criticas/(criticas+neutros) )
    pos_copom_tot.append( criticas+neutros ) 



In [ ]:
df_impacto_copom = pd.DataFrame({'pre' : pre_copom,
                                 'pos' : pos_copom,
                                 'pre_total' : pre_copom_tot,
                                 'pos_total' : pos_copom_tot,
                                 'reuniao' : [i for i in range(234,(234-87),-1)]},
                               index=hora_copom)
df_impacto_copom = df_impacto_copom.sort_index()
df_impacto_copom

In [ ]:
df_impacto_copom[['pre','pos']].plot.bar();

In [ ]:
df_impacto_copom[df_impacto_copom.index>"2020-03-01"][['pre','pos']].plot.bar();

In [ ]:
df_impacto_copom['razao'] = df_impacto_copom['pos']/df_impacto_copom['pre']

In [ ]:
df_impacto_copom['pos'].plot(linewidth=10);

In [ ]:
df_impacto_copom['razao'].plot.bar(title='razão pós/pré');

COPOM COM MAIS TWEETS

In [ ]:
df_impacto_copom['total'] = df_impacto_copom['pos_total']+df_impacto_copom['pre_total']

In [ ]:
df_impacto_copom['total'].sort_values(ascending=False)

In [ ]:
df_impacto_copom.sort_values(ascending=False, by='total')[['reuniao','total']]

In [ ]:
copom_mais_tweets = df_impacto_copom['total'].sort_values(ascending=False).index[0]

COPOM COM MAIORES RAZÕES ENTRE DEPOIS E ANTES

In [ ]:
df_impacto_copom['razao_total'] = df_impacto_copom['pos_total']/df_impacto_copom['pre_total']

In [ ]:
df_impacto_copom['razao_total'].sort_values(ascending=False)

In [ ]:
df_impacto_copom.sort_values(ascending=False, by="razao_total")[['reuniao','razao_total']]

In [ ]:
copom_maior_razao_tweets = df_impacto_copom['razao_total'].sort_values(ascending=False).index[0]

COPOM COM MAIORES CRÍTICAS

In [ ]:
df_impacto_copom['pos'].sort_values(ascending=False)

In [ ]:
df_impacto_copom.sort_values(ascending=False, by="pos")[['reuniao','pos']]

In [ ]:
copom_mais_criticas = df_impacto_copom['pos'].sort_values(ascending=False).index[0]

COPOM COM MAIOR AUMENTO DE CRITICA

In [ ]:
df_impacto_copom['razao'].sort_values(ascending=False)

In [ ]:
df_impacto_copom.sort_values(ascending=False,by='razao')

In [ ]:
df_impacto_copom[df_impacto_copom.pre>0].sort_values(ascending=False,by='razao')[['reuniao','razao']]

In [ ]:
copom_maior_razao_criticas = df_impacto_copom[df_impacto_copom.pre>0].sort_values(ascending=False,by='razao').index[0]

AVALIANDO CONTEÚDOS DOS COPOMS COM MAIORES IMPACTOS

In [ ]:
stopwords = nltk.corpus.stopwords.words('portuguese')+['copom','taxa','selic','juros','juro','básico']
mystopwords = utils.mystopwords

In [ ]:
corpus = df_copom.tweet_limpo.to_list()
corpus = utils.corrige_lema(corpus)
df_copom["tweet_limpo_corrigido"] = corpus

In [ ]:
print(copom_mais_tweets)
s_aux = df_copom["tweet_limpo_corrigido"]
str0 = str(np.datetime_as_string(np.datetime64(copom_mais_tweets), unit = 'h'))
str0_ = str(np.datetime_as_string( np.datetime64(copom_mais_tweets) - np.timedelta64(1,'h'), unit = 'h'))
str1 = str(np.datetime_as_string( np.datetime64(copom_mais_tweets) - np.timedelta64(24,'h'), unit = 'h'))
str2 = str(np.datetime_as_string( np.datetime64(copom_mais_tweets) + np.timedelta64(1,'h'), unit = 'h'))

plt.rcParams["figure.figsize"] = (16,9)
plt.rcParams.update({'font.size': 10})
fig = plt.figure()

plt.subplot(1, 2, 1)
cloud = wordcloud.WordCloud(stopwords=stopwords+mystopwords, max_font_size=50, max_words=100, background_color="white",min_word_length=4).\
generate(' '.join(s_aux[str1:str0_].to_list()).lower())
# Display the generated image:
plt.imshow(cloud, interpolation='bilinear')
plt.axis("off")
plt.title("Antes");
#plt.show()

plt.subplot(1, 2, 2)
cloud = wordcloud.WordCloud(stopwords=stopwords+mystopwords, max_font_size=50, max_words=100, background_color="white",min_word_length=4).\
generate(' '.join(s_aux[str0:str2].to_list()).lower())
# Display the generated image:
plt.imshow(cloud, interpolation='bilinear')
plt.axis("off")
plt.title("Depois");
#plt.show()

plt.savefig("imagens\\Wordcloud_copom_mais_tweets.pdf", dpi=DPI)

In [ ]:
print(copom_maior_razao_tweets)
s_aux = df_copom["tweet_limpo_corrigido"]
str0 = str(np.datetime_as_string(np.datetime64(copom_maior_razao_tweets), unit = 'h'))
str0_ = str(np.datetime_as_string( np.datetime64(copom_maior_razao_tweets) - np.timedelta64(1,'h'), unit = 'h'))
str1 = str(np.datetime_as_string( np.datetime64(copom_maior_razao_tweets) - np.timedelta64(24,'h'), unit = 'h'))
str2 = str(np.datetime_as_string( np.datetime64(copom_maior_razao_tweets) + np.timedelta64(1,'h'), unit = 'h'))

plt.rcParams["figure.figsize"] = (16,9)
plt.rcParams.update({'font.size': 10})
fig = plt.figure()

plt.subplot(1, 2, 1)
cloud = wordcloud.WordCloud(stopwords=stopwords+mystopwords, max_font_size=50, max_words=100, background_color="white",min_word_length=4).\
generate(' '.join(s_aux[str1:str0_].to_list()).lower())
# Display the generated image:
plt.imshow(cloud, interpolation='bilinear')
plt.axis("off")
plt.title("Antes");
#plt.show()

plt.subplot(1, 2, 2)
cloud = wordcloud.WordCloud(stopwords=stopwords+mystopwords, max_font_size=50, max_words=100, background_color="white",min_word_length=4).\
generate(' '.join(s_aux[str0:str2].to_list()).lower())
# Display the generated image:
plt.imshow(cloud, interpolation='bilinear')
plt.axis("off")
plt.title("Depois");
#plt.show()

plt.savefig("imagens\\Wordcloud_copom_maior_razao_tweets.pdf", dpi=DPI)

In [ ]:
print(copom_mais_criticas)
s_aux = df_copom["tweet_limpo_corrigido"]
str0 = str(np.datetime_as_string(np.datetime64(copom_mais_criticas), unit = 'h'))
str0_ = str(np.datetime_as_string( np.datetime64(copom_mais_criticas) - np.timedelta64(1,'h'), unit = 'h'))
str1 = str(np.datetime_as_string( np.datetime64(copom_mais_criticas) - np.timedelta64(24,'h'), unit = 'h'))
str2 = str(np.datetime_as_string( np.datetime64(copom_mais_criticas) + np.timedelta64(1,'h'), unit = 'h'))

plt.rcParams["figure.figsize"] = (16,9)
plt.rcParams.update({'font.size': 10})
fig = plt.figure()

plt.subplot(1, 2, 1)
cloud = wordcloud.WordCloud(stopwords=stopwords+mystopwords, max_font_size=50, max_words=100, background_color="white",min_word_length=4).\
generate(' '.join(s_aux[str1:str0_].to_list()).lower())
# Display the generated image:
plt.imshow(cloud, interpolation='bilinear')
plt.axis("off")
plt.title("Antes");
#plt.show()

plt.subplot(1, 2, 2)
cloud = wordcloud.WordCloud(stopwords=stopwords+mystopwords, max_font_size=50, max_words=100, background_color="white",min_word_length=4).\
generate(' '.join(s_aux[str0:str2].to_list()).lower())
# Display the generated image:
plt.imshow(cloud, interpolation='bilinear')
plt.axis("off")
plt.title("Depois");
#plt.show()

plt.savefig("imagens\\Wordcloud_copom_mais_criticas.pdf", dpi=DPI)

In [ ]:
print(copom_maior_razao_criticas)
s_aux = df_copom["tweet_limpo_corrigido"]
str0 = str(np.datetime_as_string(np.datetime64(copom_maior_razao_criticas), unit = 'h'))
str0_ = str(np.datetime_as_string( np.datetime64(copom_maior_razao_criticas) - np.timedelta64(1,'h'), unit = 'h'))
str1 = str(np.datetime_as_string( np.datetime64(copom_maior_razao_criticas) - np.timedelta64(24,'h'), unit = 'h'))
str2 = str(np.datetime_as_string( np.datetime64(copom_maior_razao_criticas) + np.timedelta64(1,'h'), unit = 'h'))

plt.rcParams["figure.figsize"] = (16,9)
plt.rcParams.update({'font.size': 10})
fig = plt.figure()

plt.subplot(1, 2, 1)
cloud = wordcloud.WordCloud(stopwords=stopwords+mystopwords, max_font_size=50, max_words=100, background_color="white",min_word_length=4).\
generate(' '.join(s_aux[str1:str0_].to_list()).lower())
# Display the generated image:
plt.imshow(cloud, interpolation='bilinear')
plt.axis("off")
plt.title("Antes");
#plt.show()

plt.subplot(1, 2, 2)
cloud = wordcloud.WordCloud(stopwords=stopwords+mystopwords, max_font_size=50, max_words=100, background_color="white",min_word_length=4).\
generate(' '.join(s_aux[str0:str2].to_list()).lower())
# Display the generated image:
plt.imshow(cloud, interpolation='bilinear')
plt.axis("off")
plt.title("Depois");
#plt.show()

plt.savefig("imagens\\Wordcloud_copom_maior_razao_criticas.pdf", dpi=DPI)